In [19]:
from pymongo import MongoClient
import json
import requests
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
import re
import os

# DB connexion
global client
client = MongoClient(port=27017)
global db
db = client.GeniusScrappingProject

API_BASE = "https://api.genius.com"
API_CLIENT_ACCESS_TOKEN = "w7XemCkBJW-a0eT5RZ3bjHKt9-fr-v0RYBYS4iQ_9CtY-uvbJomzCpuqoonzw6jM"


def add_songs(songs):
	if isinstance(songs, list):
		for song, _ in enumerate(songs):
			print (song)
			entry = {
				'id' : int(songs[song]['id']),
				'title' : songs[song]['title'],
				'primary_artist' : {
					'id' : songs[song]['primary_artist']['id'],
					'name' : songs[song]['primary_artist']['name'],
					'url' : songs[song]['primary_artist']['url'],
					'is_verified' : songs[song]['primary_artist']['is_verified']
					},
				'url' : songs[song]['url']
				}
			print(songs[song]['album'])
			if songs[song]['album']:
				entry['album'] = {
					'id': songs[song]['album']['id'], 
					'full_title': songs[song]['album']['full_title'], 
					'name': songs[song]['album']['name'], 
					'artist': songs[song]['album']['artist']['id']
					}
			if len(songs[song]['featured_artists']) > 0:
				featured_artists = list()
				for artist in songs[song]['featured_artists']:
					art = {
						'id' : artist['id'],
						'name' : artist['name']
						}
					featured_artists.append(art)
				entry['featured_artists'] = featured_artists
			#Step 3: Insert Artist into MongoDB via isnert_one
			db.songs.insert_one(entry)
			print('songs {} added with success'.format(songs[song]['title']))
	else:
		entry = {
			'id' : int(songs['id']),
			'title' : songs['title'],
			'primary_artist' : {
				'id' : songs['primary_artist']['id'],
				'name' : songs['primary_artist']['name'],
				'url' : songs['primary_artist']['url'],
				'is_verified' : songs['primary_artist']['is_verified']
				},
			'url' : songs['url'],
			'album': {
				'id': songs['album']['id'], 
				'full_title': songs['album']['full_title'], 
				'name': songs['album']['name'], 
				'artist': songs['album']['artist']['id']
				}
			}
		if len(songs['featured_artists']) > 0:
			featured_artists = list()
			for artist in songs['featured_artists']:
				art = {
					'id' : artist['id'],
					'name' : artist['name']
					}
				featured_artists.append(art)
		entry['featured_artists'] = featured_artists

		db.songs.insert_one(entry)
		print('songs {} added with success'.format(songs['title']))

In [20]:
def get_json(path, params=None, headers=None):
    """
    Generate Request URL and Get response object from querying the genius API

    Args:
        path (string): url to get data from
        params (optional): request parameters. Defaults to None.
        headers (optional): check if Authorization existes create one otherwise. Defaults to None.

    Returns:
        json : request result
    """
    # Generate request URL
    requrl = '/'.join([API_BASE, path])
    token = "Bearer {}".format(API_CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}
    # Get response object from querying genius api
    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()
    return response.json()

In [21]:
def get_artist_songs_id(artist_id):
    """
    Retrieve all the songs IDs of an artist

    Args:
        artist_id (Int): THe artist's ID

    Returns:
        List: all the songs that the artist is the primary one
    """
    #Get all the song id from an artist.#
    current_page = 1
    next_page = True
    songs = []  # to store final song ids

    while next_page:
        path = "artists/{}/songs/".format(artist_id)
        params = {'page': current_page}  # the current page
        data = get_json(path=path, params=params)  # get json of songs
        page_songs = data['response']['songs']
        if page_songs:
            # Add all the songs of current page
            songs += page_songs
            # Increment current_page value for next loop
            current_page += 1
            print("Page {} finished scraping".format(current_page))
            # If you don't wanna wait too long to scrape, un-comment this
            # if current_page == 2:
            #   break
        else:
            # If page_songs is empty, quit
            next_page = False

    print("Song id were scraped from {} pages".format(current_page))

    # Get all the song ids, excluding not-primary-artist songs.
    songs = [song["id"] for song in songs]
    # if song["primary_artist"]["id"] == artist_id

    return songs


In [30]:
def search(query, typee='artist'):
    """
    This is a one function to get data through the Genius API

    Args:
        typee (string) : takes one of the following values : artist - song
        query (string / int) : string when type search, int otherwise

    Returns:
        dict: the full data
    """
    if typee == 'artist' or typee == 'song':
        assert str(query).isdecimal()
        url = '{0}s/{1}'.format(typee, query)
        request = get_json(url)
        data = request['response'][typee]
    else:
        url = 'search?access_token={0}&q={1}'.format(API_CLIENT_ACCESS_TOKEN, query)
        request = get_json(url)
        data = request['response']['hits']
    return data

In [22]:
songs = get_artist_songs_id(45)

Page 2 finished scraping
Page 3 finished scraping
Page 4 finished scraping
Page 5 finished scraping
Page 6 finished scraping
Page 7 finished scraping
Page 8 finished scraping
Page 9 finished scraping
Page 10 finished scraping
Page 11 finished scraping
Page 12 finished scraping
Page 13 finished scraping
Page 14 finished scraping
Page 15 finished scraping
Page 16 finished scraping
Page 17 finished scraping
Page 18 finished scraping
Page 19 finished scraping
Page 20 finished scraping
Page 21 finished scraping
Page 22 finished scraping
Page 23 finished scraping
Page 24 finished scraping
Page 25 finished scraping
Page 26 finished scraping
Page 27 finished scraping
Page 28 finished scraping
Page 29 finished scraping
Page 30 finished scraping
Page 31 finished scraping
Page 32 finished scraping
Page 33 finished scraping
Page 34 finished scraping
Page 35 finished scraping
Page 36 finished scraping
Page 37 finished scraping
Page 38 finished scraping
Page 39 finished scraping
Page 40 finished scr

In [32]:
if isinstance(songs, list):
  for song_id in songs:
      song = search(song_id, 'song')
      print(song_id)
      entry = {
          'id' : int(song['id']),
          'title' : song['title'],
          'primary_artist' : {
              'id' : song['primary_artist']['id'],
		      'name' : song['primary_artist']['name'],
		      'url' : song['primary_artist']['url'],
	          'is_verified' : song['primary_artist']['is_verified']
		      },
		  'url' : song['url']
		  }
      if song['album']:
          entry['album'] = {
              'id': song['album']['id'], 
              'full_title': song['album']['full_title'], 
              'name': song['album']['name'], 
              'artist': song['album']['artist']['id']
              }
      if len(song['featured_artists']) > 0:
          featured_artists = list()
          for artist in song['featured_artists']:
              art = {
                  'id' : artist['id'],
                  'name' : artist['name']
                  }
              featured_artists.append(art)
          entry['featured_artists'] = featured_artists
      print (entry)
      break

4182181
None
{'id': 4182181, 'title': '12 Days of Diss-Mas', 'primary_artist': {'id': 45, 'name': 'Eminem', 'url': 'https://genius.com/artists/Eminem', 'is_verified': True}, 'url': 'https://genius.com/Eminem-12-days-of-diss-mas-lyrics'}


In [33]:
print(len(songs))

1203


In [63]:
def add_artist(artist):
    if artist['id'] == 45:
        entry = {
            'id' : int(artist['id']),
            'name' : artist['name'],
            'is_verified' : artist['is_verified'],
            'url' : artist['url'],
            'songs' : songs
            }
    else:
        entry = {
            'id' : int(artist['id']),
            'name' : artist['name'],
            'is_verified' : artist['is_verified'],
            'url' : artist['url'],
            'songs' : get_artist_songs_id(artist['id'])
            }
			#Step 3: Insert Artist into MongoDB via isnert_one
        db.artists.insert_one(entry)

def add_artists(artists):
    if isinstance(artists, list):
        for artist_id in artists:
            artist = search(artist_id)
            add_artist(artist)
            print('artists {} added with success'.format(artist['name']))
    else:
        artist = search(artists)
        add_artist(artist)
        print('artists {} added with success'.format(artist['name']))


In [64]:
add_artists([45, 56, 130, 4, 72, 59, 22])

artists Eminem added with success
Page 2 finished scraping
Page 3 finished scraping
Page 4 finished scraping
Page 5 finished scraping
Page 6 finished scraping
Page 7 finished scraping
Page 8 finished scraping
Page 9 finished scraping
Page 10 finished scraping
Page 11 finished scraping
Page 12 finished scraping
Page 13 finished scraping
Page 14 finished scraping
Page 15 finished scraping
Page 16 finished scraping
Page 17 finished scraping
Page 18 finished scraping
Page 19 finished scraping
Page 20 finished scraping
Page 21 finished scraping
Page 22 finished scraping
Page 23 finished scraping
Page 24 finished scraping
Page 25 finished scraping
Page 26 finished scraping
Page 27 finished scraping
Page 28 finished scraping
Page 29 finished scraping
Page 30 finished scraping
Page 31 finished scraping
Page 32 finished scraping
Page 33 finished scraping
Page 34 finished scraping
Page 35 finished scraping
Page 36 finished scraping
Page 37 finished scraping
Page 38 finished scraping
Page 39 fini